In [3]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import utils_v2 as utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
train = pd.read_csv('../dataset/train.csv')
test = pd.read_csv('../dataset/test.csv')
train.shape, test.shape

((1460, 81), (1459, 80))

## 1. Data Cleaning

### 1.1. Xoá dữ liệu bị khuyết nhiều (>50%)

In [5]:
train_missing_data = utils.find_missing_data(train)
train_missing_data.head()

,Count,Percent
PoolQC,1453,0.995205
MiscFeature,1406,0.963014
Alley,1369,0.937671
Fence,1179,0.807534
FireplaceQu,690,0.472603


In [6]:
# Lấy dữ ra những cột mà có lượng dữ liệu thiếu quá 50%
# Với những dữ liệu thiếu nhiều như PoolQC (>99%) hay  MiscFeature (>96%) 
# thì dù có fill bằng những giá trị khác NA đi nữa thì phần lớn data là giống nhau trên tất cả các quan sát
# do đó rất dễ có khả năng bị loại bỏ trong giai đoạn "Feature selection"
# => Bỏ trước đỡ mất công biến đổi ảnh hưởng performance
large_missing_columns = train_missing_data[train_missing_data['Percent'] > 0.5].index.to_list() 
# Xoá dữ liệu bị thiếu trong training set và test set
train.drop(large_missing_columns, axis=1, inplace=True)
test.drop(large_missing_columns, axis=1, inplace=True)

train_missing_data.drop(large_missing_columns, axis=0, inplace=True)
train_missing_data.head()

,Count,Percent
FireplaceQu,690,0.472603
LotFrontage,259,0.177397
GarageYrBlt,81,0.055479
GarageCond,81,0.055479
GarageType,81,0.055479


### 4.2 Điền các dữ liệu bị thiếu với MICE

Đây là phương pháp sử dụng các thuật toán học máy để dự đoán các dữ liệu bị thiếu.

**MICE Alogorithm cho number variable**

- Bước 1: Khởi tạo dataset, điền các giá trị NA bằng giá trị mean.
- Bước 2: Với một missing variable bất kỳ, sẽ được chọn làm **target**, các feature khác sẽ đóng vai trò làm 
  features để dự đoán missing variable đc chọn trên kia => train ra một model
- Bước 3: Dùng model được train từ Bước 2 để dự đoán các giá trị còn thiếu của missing variable đó.
- Bước 4: Lặp lại để điền các missing value khác theo cách tương tự.

**MICE Alogorithm cho category variable**

- Với category thì không thể dùng mean, mà phải dùng mode. 
- Gradient Boosting Classifier to impute the ordinal encoded data
...

In [7]:
impute_cols = train_missing_data[train_missing_data['Percent'] > 0].index.to_list()
numeric_cols = train.select_dtypes(include=['int64', 'float64']).columns
categoric_cols = train.select_dtypes(include=['object']).columns

In [8]:
train_numeric = train[numeric_cols[:-1]] #ignore SalePrice
train_categoric = train[categoric_cols]

test_numeric = test[numeric_cols[:-1]]
test_categoric = test[categoric_cols]

In [9]:
train_numeric_imp, test_numeric_imp = utils.mice_imputation_numeric(train_numeric, test_numeric)
train_categoric_imp, test_categoric_imp = utils.mice_imputation_categoric(train_categoric, test_categoric)
'''Concatenate Numeric and Categoric Training and Test set data '''
train = pd.concat([train_numeric_imp, train_categoric_imp, train['SalePrice']], axis = 1)
test = pd.concat([test_numeric_imp, test_categoric_imp], axis =1)